# PMM for 1-d Schrodinger equation

Solve 
$$H= K + c V_0\theta(|x|<R_0)$$ in periodic boundary lattice

It is only meant to be for 1D with small size $L<=100 a$.

In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch import FloatTensor
from torch.autograd import Variable
import scipy

#-----------setup lattice----------------------------------
hbarc = 197.3
L = 40
cutoff = 200.0 # MeV 
a_fm = hbarc/cutoff ;print('a = {} fm L={} fm'.format(a_fm, L*a_fm))
mass = 938.92/cutoff #l.u.
iKin = 0             # kinectic term option 

r = np.arange(L)
nx = r % L
dx = (nx +L/2.0) % L - L/2.0
dr = np.sqrt(dx**2)
qx = dx*(2*np.pi)/L  # momentum qx in range(-2pi/L,2pi/L) 
q2 = qx**2

#--------setup Hfree
def get_HKin(iKin,test=False):
    """
    construct K(i,j) matrix with approximation
    iKin=0 case, one have to use FFT. 
    
    Here we are not using sparse matrix.
    """
    if iKin==0:
        diffs = np.fft.ifft(qx**2)
        temp = np.eye(L)*0j
        for i in range(L):
            for j in range(L):
                temp[i,j] = diffs[(i-j)%L]
        Hfree = np.real(temp)
    else:
        if iKin==1: w0, w1, w2, w3 = [1.0,1.0,0.0,0.0]
        if iKin==2: w0, w1, w2, w3 = [49./36.,3./2.,3./20.,1./90.]
         
        Hfree = np.eye(L)*w0*2 # w0
        Hfree[r, (r+1)%L] = w1*(-1)
        Hfree[r, (r-1)%L] = w1*(-1)
        Hfree[r, (r+2)%L] = w2
        Hfree[r, (r-2)%L] = w2
        Hfree[r, (r+3)%L] = w3*(-1)
        Hfree[r, (r-3)%L] = w3*(-1)
        # in case of 1D, no overlap between hopping. 
        # However, in other dimension, one have to use summation. 
    
    if test:
        ee,vv = scipy.linalg.eig(Hfree)
        print('p^2  Hkin:{} '.format(np.sort(ee)))
        if iKin==0: print('p^2  exa :{} '.format(np.sort(qx**2) )) 
        if iKin>0:
            print('p^2  exa :{} '.format(np.sort(2*(w0-w1*np.cos(qx)+w2*np.cos(2*qx)-w3*np.cos(3*qx))) ))
        
    return Hfree/(2.0*mass)

a = 0.9865 fm L=39.46 fm


## Test calculation with the answer 

For a test, from Wikipedia of square wall problem, if we choose $u_0^2=m(2R_0)^2 V_0/(2\hbar^2)=20$,
with potential wall length $2R_0$ and depth $V_0$.
Solution is $E_n= 2 \hbar^2 v_n^2/(m (2R_0)^2)$ with $v_1=1.28$,$v_2=2.54$,$v_3=3.73$.

To be consistent with continuum values, one have to use small lattice spacing 
and large lattice size.
In other words, direct coordinate lattice for continuum limit is not practical
Lagrange mesh or spline interpolation may be useful. 

In [4]:
#----set potential
R0 = 3.0/a_fm # 2 fm -> l.u 
#V0 = -10.0/cutoff # 10 MeV -> l.u 
V0 = -20./mass/R0**2
print('V0={} MeV, R0={} fm'.format(V0*cutoff,R0*a_fm))

#---Wiki sol-------
vv = np.array([1.28, 2.54, 3.73])
print('E_continuum={} MeV'.format( (2*vv**2/mass/(2*R0)**2 +V0)*cutoff))

#---solve S.E. 
Hfree = get_HKin(iKin)
H_V = np.zeros((L,L))
H_V[ abs(dx)< R0,abs(dx)< R0 ] = V0
Htot = Hfree+H_V

#----get first 20 eigenvalues
#ee = scipy.linalg.eigvals(Htot)
#print(np.sort(ee)[:20]*cutoff)
#--or use torch
print('E_lattice={} MeV'.format(torch.linalg.eigvalsh(torch.Tensor(Htot))[:4]*cutoff))

V0=-92.13254472041164 MeV, R0=3.0 fm
E_continuum=[-88.35879569 -77.27248658 -60.08677268] MeV
E_lattice=tensor([-89.0333, -79.7472, -64.2924, -42.8014]) MeV


## Test PMM with square-well potential problem 

1. Prepare list of eigenvalues with several input parameter ($c_i$)
2. Prepare PMM Hermitian matrix.
3. train matrix with input eigenvaluesets. 

In [5]:
#---prepare data
clist = [0.1,0.2,0.3,0.5,0.6,0.7,0.8,0.9]
#clist = [0.1,0.3,0.5,0.8]
Hfree = get_HKin(iKin)
R0 = 3.0/a_fm # fm -> l.u 
V0 = -150.0/cutoff # MeV -> l.u 
data=[]
for c in clist:
    H_V = np.zeros((L,L))
    H_V[ abs(dx)< R0,abs(dx)< R0 ] = c*V0
    Htot = Hfree+H_V
    data.append(torch.linalg.eigvalsh(torch.Tensor(Htot)).numpy()[:2])
data = np.array(data)
print(data)    
data_tensor = torch.tensor(data)

[[-0.06351455 -0.03116331]
 [-0.13666263 -0.09742555]
 [-0.21075809 -0.16841096]
 [-0.3598199  -0.31439325]
 [-0.43453804 -0.38821527]
 [-0.50932145 -0.46231636]
 [-0.58414865 -0.5366041 ]
 [-0.65900743 -0.61102486]]


## Eigenvector Continuation test
benchmark Eigenvector Continuation at c=[0.5, 1.5]
for g.s. and 1st excited state

In [ ]:
#--prepare EC training data 
ec_evals=[]
ec_vects=[]
ec_mats =[]
for c in [0.5,1.5]:
    H_V = np.zeros((L,L))
    H_V[ abs(dx)< R0,abs(dx)< R0 ] = c*V0
    Htot = Hfree+H_V
    ee,vv = np.linalg.eigh(Htot)
    ec_mats.append(Htot)
    ec_evals.append(ee[0]) # gs case 
    ec_vects.append(vv[:,0])
    # print('test eigen:{} <H>:{}'.format(ee[0], vv[:,0]@ Htot@vv[:,0]  ) )
ec_mats = np.array(ec_mats)
ec_evals = np.array(ec_evals)
ec_vects = np.array(ec_vects)
#----consistency test of eigenvector, <H>
for i in range(2):
    print('{}: eigv={} <H>={}'.format(i,ec_evals[i], ec_vects[i,:]@ ec_mats[i] @ ec_vects[i,:]))

def EC_test(c):
    #----prediction by EC
    H_V = np.zeros((L,L))
    H_V[ abs(dx)< R0,abs(dx)< R0 ] = c*V0
    Htot = torch.Tensor(Hfree+H_V) # H(c) 
    #---construct reduced matrix and norm matrix 
    Mc = np.zeros((2,2))
    Nc = np.zeros((2,2))
    for i in range(2):
        for j in range(2):
            Mc[i,j] = np.matmul(ec_vects[i,:],np.matmul(Htot,ec_vects[j,:]))
            Nc[i,j] = np.matmul(ec_vects[i,:],ec_vects[j,:])
    ee,vv = scipy.linalg.eig(Mc,Nc)       
    return ee, vv

ee,vv=EC_test(1.5)
print('EC e={}'.format(ee[1]))

### 2d PMM
Use Pauli matrices to represent 2-d Hermitian matrices.
$$ M(c)=M_1 + c M_2, \quad M_i = \sum_{j=0,3} v^{(i)}_j \sigma_j$$
Then, train $v^{(i)}_j$.

In [ ]:
s0 = torch.tensor( [ [1.0,0.0],[0.0,1.0]])
sx = torch.tensor( [ [0.0,1.0],[1.0,0.0]])
sy = torch.tensor( [ [0.0+0j,-1j],[1j,0.0+0j]],dtype=torch.complex64)
sz = torch.tensor( [ [1.0,0.0],[0.0,-1.0]])

var1 = Variable(torch.tensor([0.5,0.5,0.5,0.5]),requires_grad=True)
var2 = Variable(torch.tensor([0.5,0.5,0.5,0.5]),requires_grad=True)
optim = torch.optim.Adam([var1,var2],lr=5.e-3)
#optim = torch.optim.SGD([var1,var2])
get_loss = nn.MSELoss(reduction='sum')

def mm_var(var):
    return s0*var[0]+sx*var[1]+sy*var[2]+sz*var[3]

train PMM : repeat until loss becomes small

In [ ]:
for ii in range(5000):
  dd = torch.zeros_like(data_tensor)
  for i,c in enumerate(clist):
      M1 = mm_var(var1)
      M2 = mm_var(var2)
      Mtot = M1 + c *M2
      dd[i] = torch.linalg.eigvalsh(Mtot)[:2]
    
  loss = get_loss(dd,data_tensor)
  optim.zero_grad()
  loss.backward()
  optim.step()
  if ii %1000 ==0:  print('epoch={} loss={}'.format(ii,loss))
print(loss)  

inference.

In [ ]:
out_t=[]
out_p=[]
c_p = np.arange(0.05,2.5,0.1)
for c in c_p:
  H_V = np.zeros((L,L))
  H_V[ abs(dx)< R0,abs(dx)< R0 ] = c*V0
  Htot = Hfree + H_V
  out_t.append(torch.linalg.eigvalsh(torch.Tensor(Htot)).numpy()[:2])
    
  with torch.inference_mode():
    M1 = mm_var(var1)
    M2 = mm_var(var2)
    Mtot = M1 + c *M2
    out_p.append(torch.linalg.eigvalsh(Mtot).numpy())

out_t = np.array(out_t)
out_p = np.array(out_p)

In [ ]:
import matplotlib.pyplot as plt

#-----plot------------------------------
plt.plot(clist,data[:,0],'g*')
plt.plot(clist,data[:,1],'g*')
plt.plot(c_p,out_t[:,0],'r--',label='true e0' )
plt.plot(c_p,out_t[:,1],'b--',label='true e1' )
plt.plot(c_p,out_p[:,0],'r',label='PMM e0' )
plt.plot(c_p,out_p[:,1],'b',label='PMM e1' )
plt.xlabel('c');plt.ylabel('E [l.u.]')
plt.legend()